# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
file = "../WeatherPy/city_csv"

city_df = pd.read_csv(file)
city_df = city_df.drop(columns="Unnamed: 0")
city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Carnarvon,-24.87,113.63,71.60,60,100,17.22,AU,1605810365
1,Jamestown,42.10,-79.24,55.00,37,1,16.11,US,1605810345
2,Upernavik,72.79,-56.15,27.48,87,92,5.73,GL,1605810366
3,Saint-Philippe,-21.36,55.77,73.40,60,90,8.05,RE,1605810366
4,Egvekinot,66.32,-179.17,25.00,95,100,6.67,RU,1605810366
...,...,...,...,...,...,...,...,...,...
535,Batemans Bay,-35.72,150.18,59.00,95,28,4.79,AU,1605810486
536,De-Kastri,51.47,140.78,15.89,84,100,6.26,RU,1605810486
537,Tougan,13.07,-3.07,83.62,18,74,7.65,BF,1605810486
538,Sazonovo,59.09,35.23,44.33,74,100,17.11,RU,1605810486


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

locations = city_df[["Lat","Lng"]].astype(float)
humidity = city_df["Humidity"].astype(float)

In [5]:


fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Drop rows with NaN values
city_df_clean = city_df.dropna()

# Max temperature lower than 80 but higher than 70
temp70 = city_df_clean["Max Temp"] > 70
temp70 = city_df_clean.loc[temp70]
temp80 = temp70["Max Temp"] < 80
nice_weather = temp70.loc[temp80]
nice_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Carnarvon,-24.87,113.63,71.60,60,100,17.22,AU,1605810365
3,Saint-Philippe,-21.36,55.77,73.40,60,90,8.05,RE,1605810366
23,‘Ewa Beach,21.32,-158.01,78.80,73,20,3.36,US,1605810371
27,Valle de Allende,26.93,-105.40,71.76,38,0,6.89,MX,1605810372
33,Saldanha,-33.01,17.94,73.40,43,0,17.22,ZA,1605810374
...,...,...,...,...,...,...,...,...,...
513,Santiago,-33.46,-70.65,73.00,49,20,9.17,CL,1605810289
521,Pangani,-5.43,38.97,78.96,83,3,10.31,TZ,1605810481
527,Nchelenge,-9.35,28.73,75.13,73,2,4.38,ZM,1605810483
530,Morón,-34.65,-58.62,77.00,36,40,8.05,AR,1605810485


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
city_lat = nice_weather["Lat"]
city_lng = nice_weather["Lng"]

lat_lng = list(zip(city_lat, city_lng))
lat_lng

target_search = "hotel"
target_radius = 5000
target_type = "lodging"

hotels = []

# Parameters dictionary
params = {
    "location": lat_lng,
    "keyword": target_search,
    "radius" : target_radius,
    "type": target_type,
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=params)

for hotel in lat_lng:
    response = requests.get(base_url, params=params)

In [22]:
# Wind speed less than 10 mph
low_wind = nice_weather["Wind Speed"] < 10
nice_weather = nice_weather.loc[low_wind]


# Zero cloudiness
zero_clouds = nice_weather["Cloudiness"] == 0
nice_weather = nice_weather.loc[zero_clouds]
hotel_df = nice_weather
nice_weather[zero_clouds]

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,Valle de Allende,26.93,-105.40,71.76,38,0,6.89,MX,1605810372
50,Riyadh,24.69,46.72,71.60,43,0,3.36,SA,1605810378
89,Mathbaria,22.29,89.96,71.73,85,0,3.22,BD,1605810387
91,Ancud,-41.87,-73.82,75.99,47,0,8.05,CL,1605810387
129,Hambantota,6.12,81.12,75.20,100,0,4.70,LK,1605810395
131,Aswān,24.09,32.91,71.37,27,0,9.04,EG,1605810396
184,Ban Nahin,18.24,104.21,73.92,82,0,2.53,LA,1605810407
273,Qui Nhon,13.77,109.23,74.14,89,0,4.18,VN,1605810428
303,Hong Kong,22.29,114.16,77.00,88,0,7.02,HK,1605810435
316,Guerrero Negro,27.98,-114.06,76.42,47,0,8.28,MX,1605810438


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
